In [1]:
import copy
from functools import partial
from typing import *

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import wandb
import wandb.apis

api = wandb.Api()

In [34]:
def load_one(run: wandb.apis.public.Run) -> Dict[str, Any]:
    if "count" in run.summary:
        task = run.config["task"]["name"]
        # results = run.summary["results"]
        # bpc = (sum(r["bpc"] for r in results) / len(run.summary["results"])
        #        if isinstance(results[0]["bpc"], float) else
        #         float("nan"))
        return dict(
            task=task,
            model_scale=run.config["model"].replace("EleutherAI/pythia-", ""),
            **run.config["sparsity"],
            bpc=run.summary["bpc"],
        )

def load_sweep(experiment_name: str, *single_runs: str) -> pd.DataFrame:
    return pd.DataFrame.from_dict(filter(None, (
        load_one(run)
        for run in api.runs(
            "research/sparse-attention", {"$or": [
                {"config.name": experiment_name},
                *({"display_name": n} for n in single_runs)
            ]}
        )
    )))

def technique(s: pd.Series) -> str:
    return "_".join(str(n) for n in [s["name"], s["strategy"], s["score"]] if not pd.isna(n))

df = (
    load_sweep("RMOE-63-perplexity-v1")
    .pipe(lambda d: d.assign(technique=d.apply(technique, axis=1)))
    [["model_scale", "technique", "k", "rank", "bpc"]]
)
(df.groupby("model_scale").apply(lambda d: d.sort_values("bpc").drop(columns="model_scale"))
 .style.format(dict(
     k=lambda x: "" if pd.isna(x) else f"{x:.0f}",
     rank=lambda x: "" if pd.isna(x) else f"{x:.0f}",
     bpc="{:.3f}",
)))